In [13]:
import os
import tensorflow as tf

In [28]:
os.environ["MLFLOW_TRACKING_URI"]=""
os.environ["MLFLOW_TRACKING_USERNAME"]=""
os.environ["MLFLOW_TRACKING_PASSWORD"]=""

In [15]:
# config enity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    path_of_model:Path
    training_data:Path
    all_params:dict
    mlflow_uri:str
    params_image_size:list
    params_batch_size:int

In [16]:
from ChestDiseasePrediction.constants import *
from ChestDiseasePrediction.utils.common import read_yaml,create_directories,save_json

In [24]:
class ConfigurationManager:
    def __init__(self,config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])
    
    def get_evaluation_config(self) -> ModelEvaluationConfig:
        model_eval_config = ModelEvaluationConfig(
            path_of_model="artifacts/model_training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/45-Hrishi/chest-cancer-prediction.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return model_eval_config

In [25]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [26]:
class ModelEvaluation:
    def __init__(self,config=ModelEvaluationConfig):
        self.config = config
    
    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def save_score(self):
        scores = {"loss":self.score[0],"accuracy":self.score[1]}
        save_json(path=Path("scores.json"),data=scores)
        
    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )
        
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        
    def model_evaluation(self):
            self.model = self.load_model(self.config.path_of_model)
            self._valid_generator()
            self.score = self.model.evaluate(self.valid_generator)
            self.save_score()

    def log_into_mlflow(self):
            mlflow.set_registry_uri(self.config.mlflow_uri)
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
            with mlflow.start_run():
                mlflow.log_params(self.config.all_params)
                mlflow.log_metrics({"loss":self.score[0],"accuracy":self.score[1]})

                if tracking_url_type_store != "file":
                    mlflow.keras.log_model(self.model,"model",registered_model_name="VGG16Model")
                else:
                    mlflow.keras.log_model(self.model,"model")

In [27]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = ModelEvaluation(eval_config)
    evaluation.model_evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-05-12 01:23:12,129:INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-05-12 01:23:12,137:INFO:common:yaml file: params.yaml loaded successfully]
[2024-05-12 01:23:12,137:INFO:common:created directory at: artifacts]
Found 102 images belonging to 2 classes.
13/13 [==============================] - 38s 3s/step - loss: 1.7994 - accuracy: 0.7843
[2024-05-12 01:23:51,555:INFO:common:json file saved at: scores.json]


2024/05/12 01:23:54 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-05-12 01:23:57,556:WARNING:save:Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\admin\AppData\Local\Temp\tmp0amhfu3i\model\data\model\assets
[2024-05-12 01:23:59,054:INFO:builder_impl:Assets written to: C:\Users\admin\AppData\Local\Temp\tmp0amhfu3i\model\data\model\assets]


c:\Users\admin\anaconda3\envs\chest_xrays\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG16Model'.
2024/05/12 01:24:42 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.


In [10]:
%pwd

'e:\\Programming\\Projects\\Deep Learning Projects\\Chest xray disease prediction\\research'

In [11]:
os.chdir("../")

In [12]:
%pwd

'e:\\Programming\\Projects\\Deep Learning Projects\\Chest xray disease prediction'